<a href="https://colab.research.google.com/github/artyomsol23/google_colab/blob/main/my_final_homework%5Breu_ml_school%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [ ]:
!git clone https://github.com/artyomsol23/google_colab.git

In [ ]:
train_data = pd.read_csv('/content/google_colab/reu_ml_school/final/train.csv')
train_data.head(10)

In [ ]:
train_data.isnull().sum()

In [ ]:
train_data.columns

In [36]:
train_data = train_data[['mileage', 'car_id', 'brand', 'transmission', 'price']]

In [ ]:
train_data.mileage

In [ ]:
train_mileage = train_data['mileage']
train_mileage = train_mileage.astype(str).str.replace('[^0-9]', '', regex=True)
train_mileage = pd.to_numeric(train_mileage, errors='coerce')
train_mileage = train_mileage.fillna(train_mileage.median())
train_mileage = train_mileage.astype('int64')
train_data['mileage'] = train_mileage

print(train_mileage)

In [ ]:
train_data.car_id

In [ ]:
car_id_series = train_data['car_id']
sequential_car_id = car_id_series.rank(method='dense')

print(sequential_car_id)

In [ ]:
train_data["car_id"] = sequential_car_id

train_data

In [ ]:
train_encoder = LabelEncoder()

train_data['brand'] = train_encoder.fit_transform(train_data['brand'])
train_data['transmission'] = train_encoder.fit_transform(train_data['transmission'])

original_classes = train_encoder.classes_

train_data

In [ ]:
test_data = pd.read_csv('/content/google_colab/reu_ml_school/final/test.csv')

test_data.head(10)

In [ ]:
test_data = test_data[['car_id', 'mileage', 'transmission', 'brand']]

test_data.isnull().sum()

In [ ]:
test_mileage = test_data['mileage']
test_mileage = test_mileage.astype(str).str.replace('[^0-9]', '', regex=True)
test_mileage = pd.to_numeric(test_mileage, errors='coerce')
test_mileage = test_mileage.fillna(test_mileage.median())
test_mileage = test_mileage.astype('int64')
test_data["mileage"] = test_mileage

print(test_mileage)

In [ ]:
new_values = {}
next_code = len(original_classes)

def test_encoder(encoder, column, next_available_code):
    global new_values, next_code
    encoded = []
    current_next_code = next_available_code
    local_new_values = {}

    for value in column:
        try:
            encoded.append(encoder.transform([value])[0])
        except ValueError:
            if value not in new_values and value not in local_new_values:
                local_new_values[value] = current_next_code
                current_next_code += 1
            encoded.append(new_values.get(value, local_new_values.get(value)))

    new_values.update(local_new_values)
    next_code = current_next_code

    return encoded, next_code

# test_encoder = train_encoder
test_data['brand'], next_code = test_encoder(train_encoder, test_data['brand'], next_code)
test_data['transmission'], _ = test_encoder(train_encoder, test_data['transmission'], next_code)

test_data

In [ ]:
X_train = train_data.drop(columns=["price"])
y_train = np.log1p(train_data["price"])

X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

rf = RandomForestRegressor(random_state=42)

params = {
    'n_estimators': [200, 300],
    'max_depth': [15, 20, None],
    'min_samples_split': [3, 5]
    }

grid_search = GridSearchCV(
    rf,
    params,
    cv=3,
    scoring='neg_mean_squared_error',
    n_jobs=-1
    )
grid_search.fit(X_train_split, y_train_split)

best_rf = grid_search.best_estimator_

val_predictions = best_rf.predict(X_val_split)

print('GridSearchCV: ', grid_search.best_params_)

In [ ]:
def calculate_rlmse(true_values, predicted_values):
    log_true = np.log1p(true_values)
    log_pred = np.log1p(predicted_values)
    rlmse = np.sqrt(mean_squared_error(log_true, log_pred))

    return rlmse

r2 = r2_score(y_val_split, val_predictions)
mae = mean_absolute_error(y_val_split, val_predictions)
mse = mean_squared_error(y_val_split, val_predictions)
rlmse = calculate_rlmse(y_val_split, val_predictions)

print('R2 Score: ', r2)
print('MAE: ', mae)
print('MSE: ', mse)
print('RLMSE: ', rlmse)

In [ ]:
X_train = train_data[['mileage', 'car_id', 'brand', 'transmission']]
X_test = test_data[['mileage', 'car_id', 'brand', 'transmission']]

print(X_train.columns.tolist())
print(X_test.columns.tolist())

In [26]:
predictions = np.expm1(best_rf.predict(X_test))

submission = pd.DataFrame(
    {"car_id": test_data["car_id"],
     "predicted_price": predictions})

submission.to_csv("submission.csv", index=False)